# Covid vs India
One of the biggest question I have been wondering is , with every adult allowed now to register, Where are vaccines slots available in india ? How can I make this decision programmitcally because trying to find information on government website is rather "sisyphus task".

So with rather self involed goal in mind, We would using publicly avaialbe data try to answer following questions

1. Latest vaccine slots per district and State in India
2. Top 5 States Running behind on schedule
3. Top 5 States Running ahead of everyone else.
4. When can I latest find slots in Delhi, Bombay, Chennai and Bangalore for people between 18-45


- toc: false
- branch: master
- badges: false,
- comments: true,
- categories: [vaccine, covid, jupyter, python],
- image: images/statistics.png,
- hide: false

Thanks to https://github.com/bhattbhavesh91/cowin-vaccination-slot-availability/blob/main/cowin-api-availability.ipynb for doing actual work, I picked up loads of stuff from there.


In [76]:
!{sys.executable} -m pip install --user install requests
!{sys.executable} -m pip install --user install altair
!{sys.executable} -m pip install --user install pandas
!{sys.executable} -m pip install --user install geopandas
#hide

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


Get All libraries in place

In [77]:
import geopandas as gpd
import pandas as pd
import requests
import json
from collections import defaultdict
from dataclasses import dataclass, asdict
import datetime
from typing import List

Lets make a data class to store our geographical and vaccination data

In [86]:
@dataclass
class District:
    district_id:str = None
    name:str = None
    state_id:int = None


@dataclass
class Session:
    session_id:str = None
    date:str = None
    available_capacity:int = None
    min_age_limit:int = None
    vaccine:str = None
    center_id:str = None
    district_id:str = None
        
@dataclass
class Center:
    center_id:str = None
    name:str = None
    state_name:str = None
    district_name:str = None
    block_name:str = None
    lat:int = None
    lng:int = None
    fee_type:str = None
    district_id:str = None

Lets call the API to get the populate geographical data

In [87]:
MOZILLA_HEADER = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
GET_DISTRICT_DATA_API_URL = "https://cdn-api.co-vin.in/api/v2/admin/location/districts/{}"
GET_APOINTMENT_DATA_API_URL = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByDistrict?district_id={}&date={}"

STATE_TO_DISTRICT_MAPPING = defaultdict(list)
DISTRICTS = defaultdict(District)
CENTERS = defaultdict(Center)
SESSIONS = defaultdict(Session)

STATE_ID_TO_STATE_NAME_MAPPING = {
    1:"Andaman and Nicobar",
    2:"",
    3:"",
    4:"",
    5:"",
    6:"",
    7:"",
    8:"",
    9:"",
    10:"",
    11:"",
    12:"",
    13:"",
    14:"",
    15:"",
    16:"",
    17:"",
    18:"",
    19:"",
    20:"",
    21:"",
    22:"",
    23:"",
    24:"",
    25:"",
    26:"",
    27:"",
    28:"",
    29:"",
    30:"",
    31:"",
    32:"",
    33:"",
    34:"",
    35:"",
    36:"",
    37:"",
    38:"",
    39:"",
    40:"",
}

for state_code in STATE_ID_TO_STATE_NAME_MAPPING.keys():
    headers = {'User-Agent': MOZILLA_HEADER}
    response = requests.get(GET_DISTRICT_DATA_API_URL.format(state_code), headers=headers)
    districts_data = json.loads(response.content)
    for district in districts_data['districts']:
        district_name = district['district_name']
        district_id = district['district_id']
        district = District(name = district_name, district_id = district_id , state_id=state_code) 
        STATE_TO_DISTRICT_MAPPING[state_code].append(district)
        DISTRICTS[district_id] = district

In [88]:
DISTRICT_DF = pd.DataFrame.from_dict([asdict(district) for district in DISTRICTS.values()])

,id,name,state_id
0,3,Nicobar,1
1,1,North and Middle Andaman,1
2,2,South Andaman,1
3,9,Anantapur,2
4,10,Chittoor,2


Lets now call actual api to get slots 

In [96]:
MAX_DAYS = 1
# district_id = 367


def get_data_fiften_days_in_future_from_today():
    base = datetime.datetime.today()
    date_list = [base + datetime.timedelta(days=x) for x in range(MAX_DAYS)]
    return [x.strftime("%d-%m-%Y") for x in date_list]

for district_id in DISTRICTS.keys():
    for INP_DATE in get_data_fiften_days_in_future_from_today():
        URL = GET_APOINTMENT_DATA_API_URL.format(district_id, INP_DATE)
        response = requests.get(URL)
        if response.ok:
            resp_json = response.json()
            if resp_json["centers"]:
                for center in resp_json["centers"]:
                    center_id = center["center_id"]
                    center_name = center["name"]
                    CENTERS[center_id] = Center(center_id=center_id,
                                                name=center_name,
                                                lat=center["lat"],
                                                lng=center["long"],
                                                district_id=district_id,
                                                state_name=center["state_name"],
                                                district_name=center["district_name"],
                                                block_name=center["block_name"],
                                                fee_type=center["fee_type"])
                    for session in center["sessions"]:
                        session_id = session["session_id"]
                        SESSIONS[session_id] = Session(session_id=session_id,
                                                       date=session["date"],
                                                       available_capacity=session["available_capacity"],
                                                       min_age_limit=session["min_age_limit"],
                                                       vaccine=session["vaccine"],
                                                       district_id=district_id,
                                                       center_id=center_id)
            else:
                print("No available slots on {}".format(INP_DATE))


No available slots on 01-05-2021
No available slots on 01-05-2021
No available slots on 01-05-2021
No available slots on 01-05-2021
No available slots on 01-05-2021
No available slots on 01-05-2021
No available slots on 01-05-2021
No available slots on 01-05-2021
No available slots on 01-05-2021
No available slots on 01-05-2021
No available slots on 01-05-2021
No available slots on 01-05-2021
No available slots on 01-05-2021
No available slots on 01-05-2021
No available slots on 01-05-2021
No available slots on 01-05-2021
No available slots on 01-05-2021
No available slots on 01-05-2021
No available slots on 01-05-2021
No available slots on 01-05-2021
No available slots on 01-05-2021
No available slots on 01-05-2021
No available slots on 01-05-2021
No available slots on 01-05-2021
No available slots on 01-05-2021
No available slots on 01-05-2021
No available slots on 01-05-2021
No available slots on 01-05-2021
No available slots on 01-05-2021
No available slots on 01-05-2021
No availab

In [100]:
CENTER_DF = pd.DataFrame.from_dict([asdict(district) for district in CENTERS.values()])
SESSION_DF = pd.DataFrame.from_dict([asdict(session) for session in SESSIONS.values()])

,id,date,available_capacity,min_age_limit,vaccine,center_id,district_id
0,98d8d5be-226f-4fd8-aac9-4ddcf6d86369,01-05-2021,48.0,45,,150815,367
1,0fd6ec79-9deb-4152-babf-013a88c6567c,02-05-2021,39.0,45,,150815,367
2,20362110-a5ea-424d-be3f-c6c1fc61134a,01-05-2021,0.0,18,,692573,367
3,a738f85a-e38b-43ce-82c6-905a4e5d7c7f,01-05-2021,25.0,45,,149982,367
4,d0b34a83-901f-4468-87f7-57d99a84e826,02-05-2021,0.0,45,,149982,367


In [ ]:
SESSION_DF.head()

In [102]:
CENTER_DF.head()

,id,name,state_name,district_name,block_name,lat,lng,fee_type,district_id
0,150815,Raipur Khurd PHC,Maharashtra,Buldhana,Buldhana,20.0,76.0,Free,367
1,692573,Buldhana DH - 2 (18-44),Maharashtra,Buldhana,Buldhana,0.0,0.0,Free,367
2,149982,Khamgaon UHC,Maharashtra,Buldhana,Khamgaon,20.0,76.0,Free,367
3,563391,Chikhli RH Campus,Maharashtra,Buldhana,Chikhli,20.0,76.0,Free,367
4,613551,Dhanwe Hospital Chikhali,Maharashtra,Buldhana,Chikhli,20.0,76.0,Paid,367


In [103]:
DISTRICT_DF.head()

,id,name,state_id
0,3,Nicobar,1
1,1,North and Middle Andaman,1
2,2,South Andaman,1
3,9,Anantapur,2
4,10,Chittoor,2


In [104]:
merged_df = pd.merge(SESSION_DF, CENTER_DF, on="center_id")

KeyError: 'center_id'